The dataset contains 36733 instances of 11 sensor measures aggregated over one hour (by means of average or sum) from a gas turbine. 
The Dataset includes gas turbine parameters (such as Turbine Inlet Temperature and Compressor Discharge pressure) in addition to the ambient variables.



Problem statement: predicting turbine energy yield (TEY) using ambient variables as features.



Attribute Information:

The explanations of sensor measurements and their brief statistics are given below.

Variable (Abbr.) Unit Min Max Mean
Ambient temperature (AT) C â€“6.23 37.10 17.71
Ambient pressure (AP) mbar 985.85 1036.56 1013.07
Ambient humidity (AH) (%) 24.08 100.20 77.87
Air filter difference pressure (AFDP) mbar 2.09 7.61 3.93
Gas turbine exhaust pressure (GTEP) mbar 17.70 40.72 25.56
Turbine inlet temperature (TIT) C 1000.85 1100.89 1081.43
Turbine after temperature (TAT) C 511.04 550.61 546.16
Compressor discharge pressure (CDP) mbar 9.85 15.16 12.06
Turbine energy yield (TEY) MWH 100.02 179.50 133.51
Carbon monoxide (CO) mg/m3 0.00 44.10 2.37
Nitrogen oxides (NOx) mg/m3 25.90 119.91 65.29

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras as k
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam_v2
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split,KFold

In [11]:
df = pd.read_csv("gas_turbines.csv")
df

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311
...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,111.61,10.400,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,111.78,10.433,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,110.19,10.483,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,110.74,10.533,6.2494,93.227


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15039 entries, 0 to 15038
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      15039 non-null  float64
 1   AP      15039 non-null  float64
 2   AH      15039 non-null  float64
 3   AFDP    15039 non-null  float64
 4   GTEP    15039 non-null  float64
 5   TIT     15039 non-null  float64
 6   TAT     15039 non-null  float64
 7   TEY     15039 non-null  float64
 8   CDP     15039 non-null  float64
 9   CO      15039 non-null  float64
 10  NOX     15039 non-null  float64
dtypes: float64(11)
memory usage: 1.3 MB


In [14]:
n_bins=2
label_encoder = LabelEncoder()
df['TEY_Label'] = label_encoder.fit_transform(pd.cut(df.TEY, n_bins, retbins=True)[0])

In [15]:
df

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX,TEY_Label
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722,0
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776,0
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468,0
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670,0
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,111.61,10.400,4.5186,79.559,0
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,111.78,10.433,4.8470,79.917,0
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,110.19,10.483,7.9632,90.912,0
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,110.74,10.533,6.2494,93.227,0


In [20]:
X = pd.concat((df.iloc[:,0:7],df.iloc[:,8:11]),axis=1,join='inner')
Y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [23]:
std_X_train = StandardScaler().fit_transform(X_train)
std_X_tst = StandardScaler().fit_transform(X_test)

In [24]:
pd.DataFrame(std_X_train).describe()

,0,1,2,3,4,5,6,7,8,9
count,1.052700e+04,1.052700e+04,1.052700e+04,1.052700e+04,1.052700e+04,1.052700e+04,1.052700e+04,1.052700e+04,1.052700e+04,1.052700e+04
mean,1.419550e-16,2.997128e-15,3.483276e-16,3.364945e-16,-4.940582e-16,-8.177412e-15,1.260636e-15,2.522074e-16,5.687823e-17,-4.875194e-16
std,1.000048e+00,1.000048e+00,1.000048e+00,1.000048e+00,1.000048e+00,1.000048e+00,1.000048e+00,1.000048e+00,1.000048e+00,1.000048e+00
min,-2.282476e+00,-4.232471e+00,-3.519588e+00,-2.756004e+00,-1.813832e+00,-5.046212e+00,-4.177349e+00,-2.000917e+00,-8.746222e-01,-3.885112e+00
25%,-8.298847e-01,-6.634794e-01,-6.874094e-01,-6.281935e-01,-5.084421e-01,-2.308800e-01,-4.266342e-01,-4.233955e-01,-4.960015e-01,-6.648438e-01
50%,5.245242e-02,-5.406316e-02,2.285600e-01,-2.071820e-02,-8.617707e-02,2.913376e-01,5.740218e-01,-7.295626e-02,-2.622312e-01,-1.483386e-01
75%,8.040117e-01,5.866052e-01,7.919550e-01,4.612043e-01,4.299379e-01,7.467599e-01,5.943268e-01,4.381766e-01,8.604582e-02,5.388117e-01
max,2.264903e+00,3.274287e+00,1.537791e+00,4.496416e+00,2.866442e+00,1.026086e+00,6.653943e-01,2.675177e+00,1.857112e+01,4.979092e+00


In [7]:
def create_model(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 10,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = adam_v2.Adam(learning_rate = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0)

# Define the grid search parameters

batch_size = [20,40]
epochs = [100,300]
learning_rate = [0.01,0.1]
dropout_rate = [0.1,0.2]
activation_function = ['relu','tanh','linear']
init = ['uniform','normal']
neuron1 = [10,16]
neuron2 = [4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init,neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(std_X_train,y_train)

# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

<ipython-input-7-2cc98272f7ba>:15: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasRegressor(build_fn = create_model,verbose = 0)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=100, init=uniform, learning_rate=0.01, neuron1=10, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=100, init=uniform, learning_rate=0.01, neuron1=10, neuron2=4, score=350742052864.000, total=  36.7s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=100, init=uniform, learning_rate=0.01, neuron1=10, neuron2=4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   36.6s remaining:    0.0s


[CV]  activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=100, init=uniform, learning_rate=0.01, neuron1=10, neuron2=4, score=356515119104.000, total=  29.4s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=100, init=uniform, learning_rate=0.01, neuron1=10, neuron2=4 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.1min remaining:    0.0s


...

[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=uniform, learning_rate=0.1, neuron1=16, neuron2=8, score=2258167371137024.000, total=  20.5s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=uniform, learning_rate=0.1, neuron1=16, neuron2=8 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=uniform, learning_rate=0.1, neuron1=16, neuron2=8, score=1693376890011648.000, total=  20.9s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.01, neuron1=10, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.01, neuron1=10, neuron2=4, score=698181615616.000, total=  20.1s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.01, neuron1=10, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=n

[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.1, neuron1=10, neuron2=8, score=1835974367641600.000, total=  20.3s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.1, neuron1=10, neuron2=8 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.1, neuron1=10, neuron2=8, score=1396699608121344.000, total=  20.9s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.1, neuron1=10, neuron2=8 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.1, neuron1=10, neuron2=8, score=1045884129247232.000, total=  21.0s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=normal, learning_rate=0.1, neuron1=10, neuron2=8 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=100, init=nor

[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=16, neuron2=4, score=34333188423680.000, total= 1.0min
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=16, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=16, neuron2=4, score=24853600010240.000, total= 1.1min
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=16, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=16, neuron2=4, score=11598481588224.000, total= 1.0min
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=16, neuron2=8 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, in

[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.1, neuron1=16, neuron2=8, score=11450226044829696.000, total= 1.3min
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=10, neuron2=4 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=10, neuron2=4, score=6454999252992.000, total= 1.2min
[CV] activation_function=relu, batch_size=20, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=10, neuron2=4 


KeyboardInterrupt: 

In [28]:
def model_Creation(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 10,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = adam_v2.Adam(learning_rate = learning_rate)
    model.compile(loss = 'mean_squared_error',optimizer = adam,metrics = ['accuracy'])
    return model


def Train_Test_Validation(X,Y):
    batch_size = 40
    epochs = 100
    learning_rate = 0.01
    dropout_rate = 0.1
    activation_function = 'relu'
    init = 'uniform'
    neuron1 = 16
    neuron2 =  4
    final_model = model_Creation(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2)
    #model_classifier = KerasClassifier(build_fn = final_model,verbose = 0,batch_size = batch_size,epochs = epochs)
    final_model.fit(X, Y,verbose = 0,batch_size = batch_size,epochs = epochs)
    scores = final_model.evaluate(X, Y)
    print("%s: %.2f%%" % (final_model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (final_model.metrics_names[0], scores[0]*100))
    return final_model

final_model = Train_Test_Validation(std_X_train, y_train)
#Train_Test_Validation(std_X_tst,y_test)

329/329 [==============================] - 0s 577us/step - loss: 0.0012 - accuracy: 0.9989
accuracy: 99.89%
loss: 0.12%


In [31]:
#Training Data
#pred_train = first_model.predict(np.array(x_train))
y_predict = final_model.predict(std_X_train)
y_predict = pd.Series([i[0] for i in y_predict])# Printing the metrics o/p is in obj dtype so converting to float so using i[0q]
#y_predict
#size = ["small","large"]
#0 for small and 1 for large

pred_train_class = pd.Series([0]*10527)
pred_train_class[[i>0.5 for i in y_predict]]= 1
pred_train_class.unique()
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(accuracy_score(y_train,pred_train_class))
print('\n',classification_report(y_train,pred_train_class))
print('\n',confusion_matrix(y_train,pred_train_class))

0.9988600740951838

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7723
           1       1.00      1.00      1.00      2804

    accuracy                           1.00     10527
   macro avg       1.00      1.00      1.00     10527
weighted avg       1.00      1.00      1.00     10527


 [[7719    4]
 [   8 2796]]


In [33]:
#Testing data
y_predict_tst = final_model.predict(std_X_tst)
y_predict_tst = pd.Series([i[0] for i in y_predict_tst])# Printing the metrics o/p is in obj dtype so converting to float so using i[0q]
#y_predict
#size = ["small","large"]
#0 for small and 1 for large

pred_train_class_tst = pd.Series([0]*4512)#len(std_X_train) is 4512 
pred_train_class_tst[[i>0.5 for i in y_predict_tst]]= 1
pred_train_class_tst.unique()
print(accuracy_score(y_test,pred_train_class_tst))
print('\n',classification_report(y_test,pred_train_class_tst))
print('\n',confusion_matrix(y_test,pred_train_class_tst))

0.9977836879432624

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3342
           1       1.00      1.00      1.00      1170

    accuracy                           1.00      4512
   macro avg       1.00      1.00      1.00      4512
weighted avg       1.00      1.00      1.00      4512


 [[3337    5]
 [   5 1165]]


In [32]:
len(std_X_tst)

4512